In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
from tools import *

signals = ['EDA', 'BVP', 'TEMP', 'HR']

In [2]:
wesad_path = r'./WESAD/WESAD/'
ppg_path = r'./PPGFieldStudyDataset/PPG_FieldStudy/'

In [4]:
def extract_E4zip(folderpath):
    for root, dir, files in os.walk(folderpath):
        for file in files:
            if '.zip' not in file and 'E4' not in file:
                continue
            filename = os.path.join(root, file)
            with zipfile.ZipFile(filename,"r") as zip_ref:
                zip_ref.extractall(filename.replace('.zip',''))
    print(filename, 'done')

In [6]:
# extract_E4zip(wesad_path)
# extract_E4zip(ppg_path)

# get label

In [4]:
label_file = r'./WESAD/WESAD/S10/S10_quest.csv'
labels = pd.read_csv(label_file)

In [5]:
labels

,# Subj;S10;;;;;;;;;;;;;;;;;;;;;;;;;
0,# ORDER;Base;Fun;Medi 1;TSST;Medi 2;bRead;fRea...
1,# START;2.5;27.53;38.42;54.3;83.17;23.39;34.58...
2,# END;22.5;34.25;45.4;66.55;90.15;24.5;36.15;7...
3,;;;;;;;;;;;;;;;;;;;;;;;;;;
4,# PANAS;2;1;3;1;1;1;1;1;1;2;1;1;2;1;3;2;1;2;1;...
5,# PANAS;2;1;3;4;1;2;1;1;1;3;1;1;3;1;2;1;2;4;1;...
6,# PANAS;1;1;2;2;1;1;1;1;1;2;1;1;1;1;1;1;1;2;1;...
7,# PANAS;4;4;3;1;3;1;1;4;2;4;1;3;2;4;4;5;1;4;4;...
8,# PANAS;2;2;1;1;1;1;1;1;1;2;1;1;1;1;2;2;1;2;1;...
9,;;;;;;;;;;;;;;;;;;;;;;;;;;
